TODO TS:

Target: Price Difference [PD]
Condition: Forecast Error [FE]

Preisdifferenz =  intraday - dayahead  
    - PD > 0: preis steigt von dayahead zu intraday
    - PD < 0: preis sinkt von dayahead zu intraday

Forecast error = actual production - predicted production
    - FE > 0: mehr produziert als predicted
    - FE < 0: weniger produziert als predicted



--> PD positive/negative given FE positive/negative [4 combinations]

--> PD high/low given FE high/low [4 combinations]
	--> investigate high-positive vs. high-negative combinations
    --> where to set the threshold?

#####

P(PD|FE) = P(PD, FE) / P(FE)

check both directions ==> consider interpretation
    --> P(A|B)
    --> P(B|A)

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
import seaborn as sns
sns.set(rc={'figure.figsize':(10, 5)})
sns.set_style("whitegrid", {'axes.grid' : False})
import datetime
import warnings
warnings.filterwarnings('ignore')
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
cRange = ['#475468', '#e7e6e6', '#4b73be', '#df8244', '#a5a5a5', '#f7c143',
          '#20406a', '#deeef8', '#2d6fc0', '#409cd4', '#5ccdd7', '#5ecb9e',
          '#a53218', '#c59933', '#f08a40', '#aa4f30', '#f7bf5d', '#d85733',
          '#728083', '#8babb4', '#7196ad', '#607397', '#8585c2', '#a687c1']
colors = ['#475468', '#e7e6e6', '#4b73be', 
          '#df8244', '#a5a5a5', '#f7c143'] 
cmapOffice = mpl.colors.ListedColormap([cRange[i] for i in range(0, 5)])
cmapBlue = mpl.colors.ListedColormap([cRange[i] for i in range(6, 11)])
cmapRed = mpl.colors.ListedColormap([cRange[i] for i in range(12, 17)])
cmapViolett = mpl.colors.ListedColormap([cRange[i] for i in range(18, 23)])

In [2]:
fullData = pd.read_csv("../00_data/data_with_calc_fields.csv", sep=";", decimal=",", index_col="datetime")
fullData.index = pd.to_datetime(fullData.index)

In [3]:
# label data >= 0 : 1; 0 otherwise
fullData["isPositiveOrNull_wind"] = fullData["wind_forecast_error"].map(lambda x: 1 if x >= 0 else 0)
fullData["isPositiveOrNull_solar"] = fullData["solar_forecast_error"].map(lambda x: 1 if x >= 0 else 0)
fullData["isPositiveOrNull_total"] = fullData["forecast_error"].map(lambda x: 1 if x >= 0 else 0)
fullData["isPositiveOrNull_priceDiff"] = fullData["price_diff"].map(lambda x: 1 if x >= 0 else 0)

# select conditional data for wind/solar/total forecastError isPositiveOrNull
data_isPositiveOrNull_wind = fullData[(fullData.isPositiveOrNull_wind == 1)]
data_isPositiveOrNull_solar = fullData[(fullData.isPositiveOrNull_solar == 1)]
data_isPositiveOrNull_total = fullData[(fullData.isPositiveOrNull_total == 1)]

# select conditional data for wind/solar/total forecastError isNegative
data_isNegative_wind = fullData[(fullData.isPositiveOrNull_wind == 0)]
data_isNegative_solar = fullData[(fullData.isPositiveOrNull_solar == 0)]
data_isNegative_total = fullData[(fullData.isPositiveOrNull_total == 0)]

# compute conditional probabilities for forecastError isPositiveOrNull
condProb_PD_isPositiveOrNull_on_FE_isPositiveOrNull_wind = data_isPositiveOrNull_wind["isPositiveOrNull_priceDiff"].sum() / data_isPositiveOrNull_wind["isPositiveOrNull_priceDiff"].count()
condProb_PD_isPositiveOrNull_on_FE_isPositiveOrNull_solar = data_isPositiveOrNull_solar["isPositiveOrNull_priceDiff"].sum() / data_isPositiveOrNull_solar["isPositiveOrNull_priceDiff"].count()
condProb_PD_isPositiveOrNull_on_FE_isPositiveOrNull_total = data_isPositiveOrNull_total["isPositiveOrNull_priceDiff"].sum() / data_isPositiveOrNull_total["isPositiveOrNull_priceDiff"].count()

# compute conditional probabilities for forecastError isNegative
condProb_PD_isPositiveOrNull_on_FE_isNegative_wind = data_isNegative_wind["isPositiveOrNull_priceDiff"].sum() / data_isPositiveOrNull_wind["isPositiveOrNull_priceDiff"].count()
condProb_PD_isPositiveOrNull_on_FE_isNegative_solar = data_isNegative_solar["isPositiveOrNull_priceDiff"].sum() / data_isPositiveOrNull_solar["isPositiveOrNull_priceDiff"].count()
condProb_PD_isPositiveOrNull_on_FE_isNegative_total = data_isNegative_total["isPositiveOrNull_priceDiff"].sum() / data_isPositiveOrNull_total["isPositiveOrNull_priceDiff"].count()

# result output
print("P(PD_posOrNull | FE_posOrNull) [wind] : " + str(condProb_PD_isPositiveOrNull_on_FE_isPositiveOrNull_wind))
print("P(PD_negative  | FE_posOrNull) [wind] : " + str(1 - condProb_PD_isPositiveOrNull_on_FE_isPositiveOrNull_wind))
print("\n")
print("P(PD_posOrNull | FE_posOrNull) [solar]: " + str(condProb_PD_isPositiveOrNull_on_FE_isPositiveOrNull_solar))
print("P(PD_negative  | FE_posOrNull) [solar]: " + str(1 - condProb_PD_isPositiveOrNull_on_FE_isPositiveOrNull_solar))
print("\n")
print("P(PD_posOrNull | FE_posOrNull) [total]: " + str(condProb_PD_isPositiveOrNull_on_FE_isPositiveOrNull_total))
print("P(PD_negative  | FE_posOrNull) [total]: " + str(1 - condProb_PD_isPositiveOrNull_on_FE_isPositiveOrNull_total))

print("\n------------------------------------\n")

print("P(PD_posOrNull | FE_negative) [wind] : " + str(condProb_PD_isPositiveOrNull_on_FE_isNegative_wind))
print("P(PD_negative  | FE_negative) [wind] : " + str(1 - condProb_PD_isPositiveOrNull_on_FE_isNegative_wind))
print("\n")
print("P(PD_posOrNull | FE_negative) [solar]: " + str(condProb_PD_isPositiveOrNull_on_FE_isNegative_solar))
print("P(PD_negative  | FE_negative) [solar]: " + str(1 - condProb_PD_isPositiveOrNull_on_FE_isNegative_solar))
print("\n")
print("P(PD_posOrNull | FE_negative) [total]: " + str(condProb_PD_isPositiveOrNull_on_FE_isNegative_total))
print("P(PD_negative  | FE_negative) [total]: " + str(1 - condProb_PD_isPositiveOrNull_on_FE_isNegative_total))

##### second try: indirect computation
# print("\n")
# conditionalData_priceDiff = fullData[(fullData.isPositiveOrNull_priceDiff == 1)]

# condProb_FE_on_PD_wind = conditionalData_priceDiff["isPositiveOrNull_wind"].sum() / conditionalData_priceDiff["isPositiveOrNull_wind"].count()
# condProb_FE_on_PD_solar = conditionalData_priceDiff["isPositiveOrNull_solar"].sum() / conditionalData_priceDiff["isPositiveOrNull_solar"].count()

# prob_positiveOrNull_wind = fullData["isPositiveOrNull_wind"].sum() / fullData["isPositiveOrNull_wind"].count()
# prob_positiveOrNull_solar = fullData["isPositiveOrNull_solar"].sum() / fullData["isPositiveOrNull_solar"].count()
# prob_positiveOrNull_priceDiff = fullData["isPositiveOrNull_priceDiff"].sum() / fullData["isPositiveOrNull_priceDiff"].count()

# secondTry_condProb_PD_on_FE_wind = (condProb_FE_on_PD_wind * prob_positiveOrNull_priceDiff) / prob_positiveOrNull_wind
# secondTry_condProb_PD_on_FE_solar = (condProb_FE_on_PD_solar * prob_positiveOrNull_priceDiff) / prob_positiveOrNull_solar

# print("Second try:")
# print("P(PD|FE) [wind] : " + str(secondTry_condProb_PD_on_FE_wind))
# print("P(PD|FE) [solar]: " + str(secondTry_condProb_PD_on_FE_solar))

P(PD_posOrNull | FE_posOrNull) [wind] : 0.36034940291906237
P(PD_negative  | FE_posOrNull) [wind] : 0.6396505970809376


P(PD_posOrNull | FE_posOrNull) [solar]: 0.46270446176243896
P(PD_negative  | FE_posOrNull) [solar]: 0.537295538237561


P(PD_posOrNull | FE_posOrNull) [total]: 0.3307033363390442
P(PD_negative  | FE_posOrNull) [total]: 0.6692966636609559

------------------------------------

P(PD_posOrNull | FE_negative) [wind] : 0.6153250773993808
P(PD_negative  | FE_negative) [wind] : 0.3846749226006192


P(PD_posOrNull | FE_negative) [solar]: 0.29296908452513487
P(PD_negative  | FE_negative) [solar]: 0.7070309154748651


P(PD_posOrNull | FE_negative) [total]: 0.6638863841298467
P(PD_negative  | FE_negative) [total]: 0.3361136158701533


# Expectations

Expectation: P(PD_negative  | FE_posOrNull) > P(PD_posOrNull  | FE_posOrNull)
    - since increased supply should lead to lower prices in intraday
Expectation: P(PD_posOrNull | FE_negative) > P(PD_negative | FE_negative)
    - since demand shortage should lead to higher prices in intraday
    
    

# Conclusion

Bot expectations are met with the exception of P(FE | FE_negative) for solar power where the probability of increasing prices in cases of demand shortages is lower than the probability of decreasing prices in the case of demand shortages.